In [40]:
import numpy as np

# Create environment

In [41]:
number_of_states = 16
number_of_terminal_states = 2
number_of_non_terminal_states = number_of_states - number_of_terminal_states

In [42]:
max_number_of_actions = 4

In [43]:
number_of_actions_per_non_terminal_state = np.repeat(a = max_number_of_actions, repeats = number_of_non_terminal_states)

In [44]:
number_of_state_action_successor_states = np.repeat(a = 1, repeats = number_of_states * max_number_of_actions)

In [45]:
number_of_state_action_successor_states = np.reshape(a = number_of_state_action_successor_states, newshape = (number_of_states, max_number_of_actions))

In [46]:
state_action_successor_state_indices = np.array([1, 0, 14, 4, 2, 1, 0, 5, 2, 2, 1, 6, 4, 14, 3, 7, 5, 0, 3, 8, 6, 1, 4, 9, 6, 2, 5, 10, 8, 3, 7, 11, 9, 4, 7, 12, 10, 5, 8, 13, 10, 6, 9, 15, 12, 7, 11, 11, 13, 8, 11, 12, 15, 9, 12, 13], dtype = np.int64)

In [47]:
state_action_successor_state_transition_probabilities = np.repeat(a = 1.0, repeats = number_of_non_terminal_states * max_number_of_actions * 1)

In [48]:
state_action_successor_state_rewards = np.repeat(a = -1.0, repeats = number_of_non_terminal_states * max_number_of_actions * 1)

In [49]:
state_action_successor_state_indices = np.reshape(a = state_action_successor_state_indices, newshape = (number_of_non_terminal_states, max_number_of_actions, 1))
state_action_successor_state_transition_probabilities = np.reshape(a = state_action_successor_state_transition_probabilities, newshape = (number_of_non_terminal_states, max_number_of_actions, 1))
state_action_successor_state_rewards = np.reshape(a = state_action_successor_state_rewards, newshape = (number_of_non_terminal_states, max_number_of_actions, 1))

In [50]:
state_value_function = np.zeros(shape = number_of_states, dtype = np.float64)
state_action_value_function = np.zeros(shape = (number_of_non_terminal_states, max_number_of_actions), dtype = np.float64)

In [51]:
policy = np.repeat(a = 1.0 / max_number_of_actions, repeats = number_of_non_terminal_states * max_number_of_actions)

In [52]:
policy = np.reshape(a = policy, newshape = (number_of_non_terminal_states, max_number_of_actions))

In [53]:
discounting_factor_gamma = 1.0
convergence_threshold = 0.001
maximum_number_of_value_estimations = 20

# Create algorithm

In [54]:
# This function estimates the value functions
def value_estimation(number_of_non_terminal_states, state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, convergence_threshold, discounting_factor_gamma, maximum_number_of_value_estimations, state_value_function, state_action_value_function):
    delta = np.finfo(np.float64).max
    number_of_value_estimations = 0

    while delta >= convergence_threshold and number_of_value_estimations < maximum_number_of_value_estimations:
        for i in range(0, number_of_non_terminal_states):
            # Cache state-value function for state i
            temp_state_value_function = state_value_function[i]

            # Update state-action value function based on successor states, transition probabilities, and rewards
            state_action_value_function[i, :] = np.squeeze(a = np.where(state_action_successor_state_indices[i, :, :] == i, state_action_successor_state_transition_probabilities[i, :, :] * (state_action_successor_state_rewards[i, :, :] + discounting_factor_gamma * temp_state_value_function), state_action_successor_state_transition_probabilities[i, :, :] * (state_action_successor_state_rewards[i, :, :] + discounting_factor_gamma * state_value_function[state_action_successor_state_indices[i, :, :]])), axis = 1)

            # Update state-value function
            state_value_function[i] = np.max(a = state_action_value_function[i, :])

            # Update delta for convergence criteria to break while loop and update policy
            delta = np.max(a = (delta, np.abs(temp_state_value_function - state_value_function[i])))
            
        number_of_value_estimations += 1
        
    return state_value_function, state_action_value_function

In [55]:
# This function greedily selects the policy based on the current value function
def greedy_policy_selection(state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, policy, discounting_factor_gamma, state_value_function):
    for i in range(0, number_of_non_terminal_states):
        # Update policy greedily from state-value function
        policy[i, :] = np.squeeze(a = state_action_successor_state_transition_probabilities[i, :, :] * (state_action_successor_state_rewards[i, :, :] + discounting_factor_gamma * state_value_function[state_action_successor_state_indices[i, :, :]]), axis = 1)

        # Save max policy value and find the number of actions that have the same max policy value
        max_policy_value = np.max(a = policy[i, :])
        max_policy_count = np.count_nonzero(a = policy[i, :] == max_policy_value)
            
        # Apportion policy probability across ties equally for state-action pairs that have the same value and zero otherwise
        policy[i, :] = np.where(policy[i, :] == max_policy_value, 1.0 / max_policy_count, 0.0)
        
    return policy

In [56]:
def value_iteration(number_of_non_terminal_states, state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, policy, convergence_threshold, discounting_factor_gamma, maximum_number_of_value_iterations, state_value_function, state_action_value_function):
    # Value estimation
    state_value_function, state_action_value_function = value_estimation(number_of_non_terminal_states, state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, convergence_threshold, discounting_factor_gamma, maximum_number_of_value_iterations, state_value_function, state_action_value_function)

    # Greedy policy selection
    policy = greedy_policy_selection(state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, policy, discounting_factor_gamma, state_value_function)
    
    return state_value_function, state_action_value_function, policy

# Run algorithm

In [57]:
print("\nInitial state value function")
print(state_value_function)

print("\nInitial state-action value function")
print(state_action_value_function)

print("\nInitial policy")
print(policy)

# Run value iteration
state_value_function, state_action_value_function, policy = value_iteration(number_of_non_terminal_states, state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, policy, convergence_threshold, discounting_factor_gamma, maximum_number_of_value_iterations, state_value_function, state_action_value_function)

# Print final results
print("\nFinal state value function")
print(state_value_function)
print("\nFinal state-action value function")
print(state_action_value_function)
print("\nFinal policy")
print(policy)


Initial state value function
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Initial state-action value function
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

Initial policy
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]

Final state value function
[-1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.  0.]

Final state-action value function
[[-3. -2. -1. -3.]
 [-4. -3. -2. -4.]
 [-4. -4. -3. -3.]
 [-3. -1. -2. -3.]
 [-4. -2. -2. -4.]
 [-3. -3. -3. -3.]
 [-3. -4. -4. -2.]
 [-4. -2. -3. -4.]
 [-3. -3. -3. -3.]
 [-2. -4. -4. -2.]
 [-2. -3. -3. 